In [ ]:
import pandas_datareader as wb
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:

# 코스피지수 크롤링
#약 3년치 데이터를 크롤링해 옴
start = datetime.datetime(2019, 8, 1)
end = datetime.datetime(2022, 8, 1)

# ^KS11 : 코스피
df_null = wb.DataReader("^KS11","yahoo",start,end)

# 결측치 제거
df = df_null.dropna()

# Close와 Adj Close는 중복되는 columns인것을 확인 함
# Adj Close열을 제거
df.drop(["Adj Close"],axis=1, inplace=True)

# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(len(df)-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# columns명을 알기 쉽게 한글로 변경
df.columns = ["최고가", "최저가" , "시작가", "종가", "거래량" , "등락"]

# 파일 저장
df.to_csv('kospi_주가데이터.csv',encoding='utf-8-sig')
df


In [ ]:
# 등락 값이 "0"인 데이터와 "1"인 데이터를 나눠서 리스트화
# 여기서 리스트화 하는 이유는 다음날 주가가 하락한 경우의 뉴스 타이틀과 상승한 뉴스타이틀을 크롤링해 분류하기 위함
price_data = pd.read_csv('kospi_주가데이터.csv')

#주가가 하락한 경우의 Date 데이터 리스트 date_0
df_0 = price_data[price_data['등락']==0]['Date']
date_0 = []
for i in range(0,len(df_0)):
    date_0.append(str(df_0.tolist()[i])[:10].replace('-',''))

#주가가 상승한 경우의 Date 데이터 리스트 date_1
df_1 = price_data[price_data['등락']==1]['Date']
date_1 = []
for i in range(0,len(df_1)):
    date_1.append(str(df_1.tolist()[i])[:10].replace('-',''))

date_1

In [ ]:
#뉴스타이틀 크롤링을 위한 모듈 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse


In [ ]:
# 팍스넷 크롤링 함수 paxnet_news_title
result_list = []
error_cnt = 0

def paxnet_news_title(dates):
    base_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(page, date)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select('ul.thumb-list li')
                for title in title_list:
                    try:
                        news_title = title.select_one('dl.text > dt').text.strip()
                        #"[문자]" 가 붙은 제거
                        #find 함수는 문자열안에 지정한 문자가 있을경우 그 위치(index)를 int로 반환해준다.
                        #지정한 문자를 찾지 못할 경우 -1 을 반환 
                        find_1=news_title.find("[")
                        find_2=news_title.find("]")

                        # find함수의 결과가 -1가 아닌 경우(문자열안에 "[문자]"가 있다.)
                        # 그 위치를 찾아 제거하고 result_list에 추가
                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        #find함수의 결과가 -1 일경우(문자열안에 "[문자]"가 없다.)
                        #바로 result_list에 추가
                        elif find_1 == -1:
                            result_list.append([news_title])
                    except:
                        error_cnt += 1
paxnet_news_title(date_0)
title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_0['주가변동'] = 0
result_list = []

paxnet_news_title(date_1)
title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_1['주가변동'] = 1
result_list = []



#팍스넷 크롤링 데이터 합쳐 csv파일로 만들기
title_df = pd.concat([title_df_0, title_df_1])
#중복 데이터 삭제 
del_title_df = title_df.drop_duplicates(['뉴스제목'])
#데이터프레임 저장 
del_title_df.to_csv('팍스넷_뉴스타이틀.csv', index=False, encoding='utf-8-sig')



In [ ]:
# 네이버 크롤링 함수 naver_news_title
result_list = []
error_cnt = 0
def naver_news_title(dates):
    base_url = 'https://finance.naver.com/news/mainnews.naver?date={}&page={}'
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(date,page)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select('#contentarea_left > div.mainNewsList > ul > li > dl')
                for title in title_list:
                    try:
                        news_title = title.select_one('dd.articleSubject').text.strip()

                        find_1=news_title.find("[")
                        find_2=news_title.find("]")

                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                        
                            news_title.strip(slice_news_title)
                            result_list.append([news_title.strip(slice_news_title).strip()])

                        elif find_1 == -1:
                            result_list.append([news_title])

                    except AttributeError:
                        news_title = title.select_one('dt.articleSubject').text.strip()
                        find_1=news_title.find("[")
                        find_2=news_title.find("]")
                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                        
                            news_title.strip(slice_news_title)
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        elif find_1 == -1:
                            result_list.append([news_title])
                        
                    except:
                        error_cnt += 1

naver_news_title(date_0)
title_df_2 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_2['주가변동'] = 0
result_list = []

naver_news_title(date_1)
title_df_3 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_3['주가변동'] = 1
result_list = []

#네이버 크롤링 데이터 합쳐 csv파일로 만들기
title_df2 = pd.concat([title_df_2, title_df_3])
#중복 데이터 삭제 
del_title_df2 = title_df2.drop_duplicates(['뉴스제목'])
title_df2.to_csv('네이버_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
title_df2


In [ ]:
#뉴스데이터 하나로 합치기 
naver_data = pd.read_csv('네이버_뉴스타이틀.csv')
paxnet_data = pd.read_csv('팍스넷_뉴스타이틀.csv')
all_title = pd.concat([naver_data, paxnet_data])
all_title.to_csv('paxnet_naver.csv')